In [1]:
# setup - graphical libraries
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
from IPython.display import Image
from IPython.display import display

# <font color='blue' size=8px> Boosting </font>

Boosting (Schapire and Freund 2012) is a greedy algorithm for fitting adaptive basis-function where the weights are generated by an algorithm called a weak
learner or a base learner. The algorithm works by applying the weak learner sequentially to weighted versions of the data, where more weight is given to examples that were misclassified by earlier rounds.
This weak learner can be any classification or regression algorithm. In 1998, the late Leo Breiman called boosting, where the weak learner is a shallow
decision tree, the “best off-the-shelf classifier in the world” (Hastie et al. 2009, p340). (Reference: K. Murphy - "Machine Learning - A Probilistic Perspective", page 554.)


## How Does Adaptive Boosting Work?

We can understand the working of the AdaBoost algorithm in step by step manner as going deep into the work, we can see there are multiple basic steps which this algorithm follows. Let’s take a look at these steps.

**Step 1:** When the algorithm is given data, it starts by Assigning equal weights to all training examples in the dataset. These weights represent the importance of each sample during the training process.

**Step 2:** Here, this algorithm iterates with a few algorithms for a specified number of iterations (or until a stopping criterion is met). The algorithm trains a weak classifier on the training data. Here the weak classifier can be considered a model that performs slightly better than random guessing, such as a decision stump (a one-level decision tree).

**Step 3:** During each iteration, the algorithm trains the weak classifier on given training data with the current sample weights. The weak classifier aims to minimize the classification error, weighted by the sample weights.

**Step 4:** After training the weak classifier, the algorithm calculates classifier weight based on the errors of the weak classifier. A weak classifier with a lower error receives a higher weight.

**Step 4:** Once the calculation of weight completes, the algorithm updates sample weights, and the algorithm gives assigns higher weights to misclassified examples so that more importance in subsequent iterations can be given.

**Step 5:** After updating the sample weights, they are normalized so that they sum up to 1 and Combine the predictions of all weak classifiers using a weighted majority vote. The weights of the weak classifiers are considered when making the final prediction.

**Step 6:** Finally, Steps 2–5 are repeated for the specified number of iterations (or until the stopping criterion is met), with the sample weights updated at each iteration. The final prediction is obtained by aggregating the predictions of all weak classifiers based on their weights.

The below pseudocode can be helpful in understanding the working of the AdaBoost algorithm.

```pseudo
Initialize sample weights for each training example
For each iteration:
Train a weak classifier using the current sample weights
Calculate the error of the weak classifier
Calculate the weight of the weak classifier based on the error
Update the sample weights based on the weak classifier's performance
Normalize the sample weights
End the iterations
Combine the weak classifiers using a weighted majority vote.
```

Reference: https://medium.com/@datasciencewizards/understanding-the-adaboost-algorithm-2e9344d83d9b

# <font color='blue' size=8px> Gradient Boosting </font>

<font size=5px> Main Idea:

Assume you have an regressor $F$ and, for the observation $x_i$ we make the prediction $F(x_i)$. To improve the predictions, we can regard $F$ as a 'weak learner' and therefore train a decision tree (we can call it $h$) where the new output is $y_i-F(x_i)$. So, the new predictor is trained on the residuals of the previous one. Thus, there are increased chances that the new regressor

$$\large F + h$$

is better than the old one, $F.$

Main task: implement this idea in an algorithm and test it on real data sets.


<figure>
<center>
<img src='https://i.imgur.com/K2RC0le.png'width='400px'/>
<figcaption>Computational Diagram for Gradient Boosting</figcaption></center>
</figure>


In gradient boosting for classification, the concept is very similar to the one used in regression, but instead of predicting residuals (numerical differences between predicted and actual values), we work with probabilities and classes.

### 1. **Initialize with a Base Classifier:**
   
   The algorithm starts by making an initial prediction for each instance in the training data. This is usually a constant prediction, such as the mean of the target variable, but in classification, it might be based on class probabilities. For binary classification, this could mean predicting the probability of the positive class (e.g., 0.5 for a balanced dataset).

### 2. **Compute Pseudo-Residuals:**
   
   Instead of computing residuals (differences between actual values and predicted values as in regression), we compute **pseudo-residuals**. In classification, this involves computing the difference between the true class label (often 0 or 1 for binary classification) and the predicted probability of the class.

   For a binary classification problem, if the predicted probability for class 1 is $ p_i $ and the true label is $ y_i $, the pseudo-residual is:
   $$
   r_i = y_i - p_i
   $$
   In other words, it's the difference between the true label and the predicted probability.

### 3. **Fit a New Weak Learner (Tree):**
   
   The algorithm then fits a new weak learner (usually a decision tree) to predict the pseudo-residuals. This tree is trained to focus on the areas where the current model performs poorly (i.e., where the errors or pseudo-residuals are large).

### 4. **Update the Model:**
   
   The predictions from the new tree are used to update the overall model. Instead of directly adding these predictions to the existing model (as in regression), we adjust the predictions in a way that incorporates the probabilities and the contribution from the new tree. Specifically, in binary classification, we update the log-odds of the predicted class probabilities:
   $$
   \text{logit}(p_i^{\text{new}}) = \text{logit}(p_i^{\text{old}}) + \alpha f(x_i)
   $$
   where $ f(x_i) $ is the prediction from the new tree, and $ \alpha $ is the learning rate.

   The updated predicted probability for class 1 is then:
   $$
   p_i^{\text{new}} = \frac{1}{1 + e^{-\text{logit}(p_i^{\text{new}})}}
   $$

### 5. **Repeat the Process:**
   
   Steps 2 through 4 are repeated for a set number of iterations or until the model's performance converges. Each iteration adds a new tree that attempts to correct the errors of the previous trees.

### 6. **Final Prediction:**
   
   After all iterations, the final prediction for each instance is typically the class with the highest probability. For binary classification, if the final probability $ p_i $ for class 1 is greater than 0.5, the predicted class is 1; otherwise, it’s 0.

### For Multiclass Classification:
   In multiclass classification, the process is extended by using a one-vs-all or softmax approach to predict probabilities for each class. The algorithm works similarly, but pseudo-residuals are computed for each class, and the model is trained to improve predictions for all classes simultaneously.

In summary, the key difference between regression and classification in gradient boosting is that, in classification, the model focuses on improving class probabilities, often in log-odds space, rather than directly predicting residuals as in regression. The weak learners (trees) are trained to reduce the classification error iteratively by focusing on pseudo-residuals related to the predicted probabilities.

In [1]:
# computational libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import scipy.stats as stats
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy.interpolate import interp1d, RegularGridInterpolator, griddata, LinearNDInterpolator, NearestNDInterpolator
from math import ceil
from scipy import linalg
# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [2]:
scale = StandardScaler()

## Kernels

In [3]:
# Gaussian Kernel
def Gaussian(w):
  return np.where(w>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*w**2))

# Tricubic Kernel
def Tricubic(w):
  return np.where(w>1,0,70/81*(1-w**3)**3)

# Quartic Kernel
def Quartic(w):
  return np.where(w>1,0,15/16*(1-w**2)**2)

# Epanechnikov Kernel
def Epanechnikov(w):
  return np.where(w>1,0,3/4*(1-w**2))

## Function Definitions

We define all the useful functions we need.

- we need a distance function

- we need the locally weighted regression for predicting the train data

- we need an encapsulation for SkLearn

In [ ]:
# here we have a function that computes the Euclidean distance between all the observations in u, and v
def dist(u,v):
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

In [ ]:
def lw_ag_md(x, y, xnew,f=2/3,iter=3, intercept=True):

  n = len(x)
  r = int(ceil(f * n))
  yest = np.zeros(n)

  if len(y.shape)==1: # here we make column vectors
    y = y.reshape(-1,1)

  if len(x.shape)==1:
    x = x.reshape(-1,1)

  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x

  h = [np.sort(np.sqrt(np.sum((x-x[i])**2,axis=1)))[r] for i in range(n)]
  # dist(x,x) is always symmetric
  w = np.clip(dist(x,x) / np.array(h), 0.0, 1.0)
  # note that w is a square matrix and in Python arithmetic operations such as
  # w**3 or 1-w**3 are performed element-wise
  #w = (1-w**3)**3 # a Tricubic kernel
  w = Epanechnikov(w)

  #Looping through all X-points
  delta = np.ones(n)
  for iteration in range(iter):
    for i in range(n):
      W = np.diag(delta).dot(np.diag(w[i,:]))
      # when we multiply two diagonal matrices we get also a diagonal matrix
      b = np.transpose(x1).dot(W).dot(y)
      A = np.transpose(x1).dot(W).dot(x1)
      ##
      A = A + 0.0001*np.eye(x1.shape[1]) # if we want L2 regularization for solving the system
      beta = linalg.solve(A, b)

      #beta, res, rnk, s = linalg.lstsq(A, b)
      yest[i] = np.dot(x1[i],beta.ravel())

    residuals = y.ravel() - yest
    s = np.median(np.abs(residuals))

    delta = np.clip(residuals / (6.0 * s), -1, 1)

    delta = (1 - delta ** 2) ** 2

  # here we are making predictions for xnew by using an interpolation and the predictions we made for the train data
  if x.shape[1]==1:
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
    output = f(xnew)
  else:
    output = np.zeros(len(xnew))
    for i in range(len(xnew)):
      ind = np.argsort(np.sqrt(np.sum((x-xnew[i])**2,axis=1)))[:r]
      pca = PCA(n_components=3)
      x_pca = pca.fit_transform(x[ind])
      tri = Delaunay(x_pca,qhull_options='QJ Pp')
      f = LinearNDInterpolator(tri,yest[ind])
      output[i] = f(pca.transform(xnew[i].reshape(1,-1)))
      # the output may have NaN's where the data points from xnew are outside the convex hull of X

  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,yest.ravel())
    # output[np.isnan(output)] = g(X[np.isnan(output)])
    output[np.isnan(output)] = g(xnew[np.isnan(output)])
  return output

In [ ]:
def lowess(x, y, xnew,kernel=Gaussian,tau=0.02,iter=1, intercept=True):

  n = len(x)

  yest = np.zeros(n)

  if len(y.shape)==1: # here we make column vectors
    y = y.reshape(-1,1)

  if len(x.shape)==1:
    x = x.reshape(-1,1)

  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x


  # dist(x,x) is always symmetric
  w = dist(x,x)


  #Looping through all X-points
  delta = np.ones(n)
  for iteration in range(iter):
    for i in range(n):
      W = np.diag(delta).dot(kernel(w[i,:]/(2*tau)).ravel())
      # when we multiply two diagonal matrices we get also a diagonal matrix
      b = np.transpose(x1).dot(np.diag(W)).dot(y)
      A = np.transpose(x1).dot(np.diag(W)).dot(x1)
      ##
      A = A + 0.0001*np.eye(x1.shape[1]) # if we want L2 regularization for solving the system
      beta = linalg.solve(A, b)

      #beta, res, rnk, s = linalg.lstsq(A, b)
      yest[i] = np.dot(x1[i],beta.ravel())

    residuals = y.ravel() - yest
    s = np.median(np.abs(residuals))

    delta = np.clip(residuals / (6.0 * s), -1, 1)

    delta = (1 - delta ** 2) ** 2

  # here we are making predictions for xnew by using an interpolation and the predictions we made for the train data
  if x.shape[1]==1:
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
    output = f(xnew)
  else:
    output = np.zeros(len(xnew))
    for i in range(len(xnew)):
      w = np.diag(kernel(dist(x,xnew[i])/(2*tau)).ravel())
      # model = LinearRegression()
      # model.fit(w.dot(x),w.dot(yest))
      # output[i] = model.predict(xnew[i].reshape(1,-1))

      output[i] = np.sum(w.dot(yest))/np.trace(w)
      # the output may have NaN's where the data points from xnew are outside the convex hull of X

  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,yest.ravel())
    # output[np.isnan(output)] = g(X[np.isnan(output)])
    output[np.isnan(output)] = g(xnew[np.isnan(output)])
  return output

In [ ]:
# testing a different version of lowess
yhat = lowess(xtrain,ytrain,xtest,kernel=Epanechnikov,tau=0.2,iter=1)
mse(ytest,yhat)

<ipython-input-133-9dd72b8600c5>:57: RuntimeWarning: invalid value encountered in double_scalars
  output[i] = np.sum(w.dot(yest))/np.trace(w)


190.50550599947195

In [ ]:
yhat2 = lw_ag_md(xtrain,ytrain,xtest,f=25/len(xtrain),iter=1)

In [ ]:
mse(ytest,yhat2)

59.17453576519482

In [ ]:
model = Lowess_AG_MD

## Scikit-Learn Compliant Functions

In [ ]:
class Lowess_AG_MD:
    def __init__(self, f = 1/10, iter = 3,intercept=True):
        self.f = f
        self.iter = iter
        self.intercept = intercept

    def fit(self, x, y):
        f = self.f
        iter = self.iter
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        f = self.f
        iter = self.iter
        intercept = self.intercept
        return lw_ag_md(x, y, x_new, f, iter, intercept) # this is actually our defined function of Lowess

    def get_params(self, deep=True):
    # suppose this estimator has parameters "f", "iter" and "intercept"
        return {"f": self.f, "iter": self.iter,"intercept":self.intercept}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

## The Boosted Regressor

In [ ]:
def boosted_lwr(x, y, xnew, f=1/3,iter=2,intercept=True):
  # we need decision trees
  # for training the boosted method we use x and y
  model1 = Lowess_AG_MD(f=f,iter=iter,intercept=intercept) # we need this for training the Decision Tree
  model1.fit(x,y)
  residuals1 = y - model1.predict(x)
  model2 = Lowess_AG_MD(f=f,iter=iter,intercept=intercept)
  #model2 = RandomForestRegressor(n_estimators=200,max_depth=9)
  model2.fit(x,residuals1)
  output = model1.predict(xnew) + model2.predict(xnew)
  return output

In [ ]:
data = pd.read_csv('drive/MyDrive/Data Sets/concrete.csv')
data

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [ ]:
x = data.loc[:,'cement':'age'].values
y = data['strength'].values

In [ ]:
xtrain, xtest, ytrain, ytest = tts(x,y,test_size=0.3,shuffle=True,random_state=123)

In [ ]:
xtrain = scale.fit_transform(xtrain)
xtest = scale.transform(xtest)

In [ ]:
yhat = boosted_lwr(xtrain,ytrain,xtest,f=25/len(xtrain),iter=1,intercept=True)

In [ ]:
mse(ytest,yhat)

57.73151261936397

In [ ]:
import xgboost

In [ ]:
model_xgboost = xgboost.XGBRFRegressor(n_estimators=200,max_depth=7)

In [ ]:
model_xgboost.fit(xtrain,ytrain)
mse(ytest,model_xgboost.predict(xtest))

32.63930815840025

In [ ]:
mse(ytest,yhat)

57.73151261936397

## Test a Complete K-Fold CV

In [ ]:
mse_lwr = []
mse_rf = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_rf = RandomForestRegressor(n_estimators=200,max_depth=5)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  yhat_lw = boosted_lwr(xtrain,ytrain,xtest,f=25/len(xtrain),iter=1,intercept=True)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 56.670304257102146
The Cross-validated Mean Squared Error for Random Forest is : 45.70628661801278


## Polynomial Features

This allows for more polynomially engineered features in the data. Let's see if results improve.

In [ ]:
poly = PolynomialFeatures(degree=2)
scale = StandardScaler()
pipe = Pipeline([['zscores',scale],['Poly',poly]])

In [ ]:
mse_lwr = []
mse_rf = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_rf = RandomForestRegressor(n_estimators=200,max_depth=5)
i = 1
for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x[idxtest]
  xtrain = pipe.fit_transform(xtrain)
  xtest = pipe.transform(xtest)

  yhat_lw = boosted_lwr(xtrain,ytrain,xtest,f=25/len(xtrain),iter=1,intercept=True)

  # model_rf.fit(xtrain,ytrain)
  # yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  print('MSE Fold '+str(i)+' : '+str(mse(ytest,yhat_lw)))
  i += 1
  # mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
# print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))

MSE Fold 1 :63.80053774991302
MSE Fold 2 :47.73682100017316
MSE Fold 3 :43.12602069919888
MSE Fold 4 :37.937798704824715
MSE Fold 5 :38.87599271164445
MSE Fold 6 :44.95975550888706
MSE Fold 7 :57.703227331401564
MSE Fold 8 :56.35719574743018
MSE Fold 9 :88.60327031261076
MSE Fold 10 :58.216548891120496
The Cross-validated Mean Squared Error for Locally Weighted Regression is : 53.73171686572043
